In [103]:
import cv2 as cv
import numpy as np
import time
from skimage.draw import random_shapes
from skimage.measure import compare_ssim as ssim
# import astarsearch
import traversal

In [104]:
class Node():
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position
        self.g = 0
        self.h = 0
        self.f = 0
    def __eq__(self, other):
        return self.position == other.position

def astar(maze, start, end):
    
    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:

        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1] # Return reversed path

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]: # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue

            # Add the child to the open list
            open_list.append(child)


def highlightedpath(start,end,gray,img):
    path = astar(gray, start, end)
    path_arr = np.asarray(path)
    print(path_arr)
    m,n=path_arr.shape
    if path is not None:
        for i in range (m-1):
            x1 = path[i][0]
            y1 = path[i][1]
            window_name = 'Image'
            end1 = path[i+1][0]
            end2 = path[i+1][1]
            start = (x1,y1)
            end = (end1,end2)
            color = (255, 255, 0) 
            thickness = 1
            img = cv.line(img, start, end, color, thickness)
    return img


In [105]:
img = cv.imread('test.png')
cv.imshow('world',img)
cv.waitKey(0)
cv.destroyAllWindows()

In [106]:
res = img
cv.imshow('world',res)
cv.waitKey(0)
cv.destroyAllWindows()

In [107]:
gray = cv.cvtColor(res,cv.COLOR_BGR2GRAY)
cv.imshow('Gray World',gray)
cv.waitKey(0)
cv.destroyAllWindows()

In [108]:
gray2 = gray

In [109]:
def CircleDetector(grey):
    for x in range (128):
        for y in range (128):
            if grey[x][y] != 255.0:
                grey[x][y] = 0

    grey = cv.medianBlur(grey, 3)
    rows = grey.shape[0]
    circles = cv.HoughCircles(grey, cv.HOUGH_GRADIENT, 1,rows/10000,param1=70, param2=3,minRadius=2, maxRadius=5)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            center = (i[0], i[1])
            cv.circle(res, center, 1, (0, 100, 100), 1)                        
            radius = i[2]
            cv.circle(res, center, radius, (255, 0, 255), 1)    
    return circles

In [110]:
circle_cood = CircleDetector(gray)
#these are co ordinates of the circles

In [111]:
def WorldStandardizer(gray):
    for x in range(128):
        for y in range(128):
            if gray[x][y] > 0.0 and gray[x][y] < 255.0:
                gray[x][y] = 1
            elif gray[x][y] == 255.0:
                gray[x][y] = 0
                
    return gray

In [112]:
grey1 = WorldStandardizer(gray)

In [113]:
grey2 = WorldStandardizer(gray2)

In [114]:
start = (circle_cood[0,0,0],circle_cood[0,0,1])
end = (circle_cood[0,1,0],circle_cood[0,1,1])

In [115]:
nav_img = highlightedpath(start,end,gray,img)

[[26 16]
 [27 17]
 [28 18]
 [29 19]
 [30 20]
 [31 21]
 [32 22]
 [33 23]
 [34 24]
 [35 25]
 [36 26]
 [37 27]
 [38 28]
 [39 29]
 [40 30]
 [41 31]
 [42 32]
 [43 33]
 [44 34]
 [45 35]
 [46 36]
 [47 37]
 [48 38]
 [49 38]
 [50 38]
 [51 38]
 [52 38]
 [53 38]
 [54 38]
 [55 38]
 [56 38]
 [57 38]
 [58 38]
 [59 38]
 [60 38]
 [61 38]
 [62 38]
 [63 38]
 [64 38]
 [65 38]
 [66 38]
 [67 38]
 [68 38]
 [69 38]
 [70 38]
 [71 38]
 [72 38]
 [73 38]
 [74 38]
 [75 38]
 [76 38]
 [77 38]
 [78 38]
 [79 38]
 [80 38]
 [81 38]
 [82 38]
 [83 38]
 [84 38]
 [85 38]
 [86 38]
 [87 38]
 [88 38]
 [89 38]
 [90 38]]


In [116]:
cv.imshow('Navigated Image',nav_img)
cv.waitKey(0)
cv.destroyAllWindows()